# Counterfactuals generation for classification

We show how to generate counterfactuals using the code in this repository

In [ ]:
import sys
from pathlib import Path
import joblib
import json
from src import load_dataset, Counterfactuals

PROJECT_ROOT = Path().resolve().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

We start by loading sample data from scikit-learn. For classification, we use the adult dataset, trying to predict whether an individual's income will be above or below $50k/yr.

The code that loads this data set can be found in `src/data_loader/data.py` and it preprocesses the data, mostly encoding categorical values.

OrdinalEncoding is preferred in order not to increase the amount of columns too much. Only the column `sex` has been one-hot encoded and both resulting columns, `sex_Male` and `sex_Female` preserved in the data. This choice has been made to showcase how counterfactuals can be generated while ensuring one-hot encoding is preserved. We will see in more detail how to enforce co-mutation. That is, say we start with `sex_Male=1` and `sex_Female=0` and the counterfactual mutates `sex_Male` to 0, we need to be sure `sex_Female` becomes 1.

In [2]:
_, X, y = load_dataset("adult", preprocess=True, include_description=True)

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')
Stored categorical mappings in: artifacts/categorical_encodings.json
{'age': 'Age of the individual',
 'capital-gain': 'Capital gain in USD',
 'capital-loss': 'Capital loss in USD',
 'education': 'Highest level of education achieved',
 'education-num': 'Education level as an integer',
 'fnlwgt': 'Final weight — estimation of census population',
 'hours-per-week': 'Average working hours per week',
 'marital-status': 'Marital status',
 'native-country': 'Country of origin',
 'occupation': 'Occupation',
 'race': 'Race',
 'relationship': 'Relationship status',
 'sex': 'Gender',
 'target': "Income level, '>50K' or '<=50K'",
 'workclass': 'Represents the employment status'}


The preprocessed data:

In [3]:
X

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male
0,25,7,0.0,0.0,40.0,3,4,3,2,38,0.0,1.0
1,38,9,0.0,0.0,50.0,3,2,0,4,38,0.0,1.0
2,28,12,0.0,0.0,40.0,1,2,0,4,38,0.0,1.0
3,44,10,7688.0,0.0,40.0,3,2,0,2,38,0.0,1.0
4,18,10,0.0,0.0,30.0,3,4,3,4,38,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,12,0.0,0.0,38.0,3,2,5,4,38,1.0,0.0
48838,40,9,0.0,0.0,40.0,3,2,0,4,38,0.0,1.0
48839,58,9,0.0,0.0,40.0,3,6,4,4,38,1.0,0.0
48840,22,9,0.0,0.0,20.0,3,4,3,4,38,0.0,1.0


raw data is currently stored in `_` as we don't need it for the demo.

The model is a `XGBClassifier`. No hyperparameter tuning has been done as an optimal model is not necessary for this demo.

In [5]:
ARTIFACT_DIR = Path("../artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)

model = joblib.load(ARTIFACT_DIR / "classification_model.pkl")

Just for completeness, here are the training metrics for the model:

In [6]:
model_metrics = Path(ARTIFACT_DIR / "classification_report.json")

with open(model_metrics) as f:
    model_metrics = json.load(f)
    
model_metrics

{'0': {'precision': 0.8959938366718028,
  'recall': 0.9396714247239429,
  'f1-score': 0.9173130011831208,
  'support': 3713.0},
 '1': {'precision': 0.7739656912209889,
  'recall': 0.6544368600682594,
  'f1-score': 0.7092001849283402,
  'support': 1172.0},
 'accuracy': 0.8712384851586489,
 'macro avg': {'precision': 0.8349797639463958,
  'recall': 0.7970541423961012,
  'f1-score': 0.8132565930557305,
  'support': 4885.0},
 'weighted avg': {'precision': 0.8667170738328358,
  'recall': 0.8712384851586489,
  'f1-score': 0.8673829662495275,
  'support': 4885.0}}

We see the model is definitely biased towards the class with larger support (0-class). This can be improved with some hyperparameter tuning, up-/down-sampling and/or regularization. Again, not the scope of this demo.

What might be useful for interpreting results though is having a look at categorical encodings:

In [7]:
ENCODING_PATH = Path(ARTIFACT_DIR / "categorical_encodings.json")

with open(ENCODING_PATH) as f:
    CAT_ENCODINGS = json.load(f)

for key in CAT_ENCODINGS.keys():
    print(CAT_ENCODINGS[key])

{'Federal-gov': 0, 'Local-gov': 1, 'Never-worked': 2, 'Private': 3, 'Self-emp-inc': 4, 'Self-emp-not-inc': 5, 'State-gov': 6, 'Without-pay': 7}
{'Divorced': 0, 'Married-AF-spouse': 1, 'Married-civ-spouse': 2, 'Married-spouse-absent': 3, 'Never-married': 4, 'Separated': 5, 'Widowed': 6}
{'Husband': 0, 'Not-in-family': 1, 'Other-relative': 2, 'Own-child': 3, 'Unmarried': 4, 'Wife': 5}
{'Amer-Indian-Eskimo': 0, 'Asian-Pac-Islander': 1, 'Black': 2, 'Other': 3, 'White': 4}
{'Cambodia': 0, 'Canada': 1, 'China': 2, 'Columbia': 3, 'Cuba': 4, 'Dominican-Republic': 5, 'Ecuador': 6, 'El-Salvador': 7, 'England': 8, 'France': 9, 'Germany': 10, 'Greece': 11, 'Guatemala': 12, 'Haiti': 13, 'Holand-Netherlands': 14, 'Honduras': 15, 'Hong': 16, 'Hungary': 17, 'India': 18, 'Iran': 19, 'Ireland': 20, 'Italy': 21, 'Jamaica': 22, 'Japan': 23, 'Laos': 24, 'Mexico': 25, 'Nicaragua': 26, 'Outlying-US(Guam-USVI-etc)': 27, 'Peru': 28, 'Philippines': 29, 'Poland': 30, 'Portugal': 31, 'Puerto-Rico': 32, 'Scotland'

## Generating counterfactuals

Typically, one would have a model response and may ask the question: "what would need to be different in order to get a specific result?"

In this case, trying to predict whether someone is above or below a certain income threshold, we may ask "what would this person need to do, according to the model, in order to earn more/less?"

The answer is what counterfactuals provide.

We start with a data instance we want to reverse:

In [7]:
instance = X.iloc[0:1].copy()
instance_outcome = y.iloc[0:1]
instance.loc[:, 'outcome'] = instance_outcome.values

In [8]:
instance

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
0,25,7,0.0,0.0,40.0,3,4,3,2,38,0.0,1.0,0


We see this person would not earn more than $50k/yr (outcome=0)

So, what would it need to be different for him to earn more?

We initialize the `Counterfactuals` class:

In [9]:
cf = Counterfactuals(X=X, y=y, model=model)

Now, there are currently two ways provided  in this repository in order to explore counterfactuals.

First up, it might be that there are similar instances of data that already have a different outcome. This would be what is called a prototype: an existing data instance with the desired model outcome.

We can then look into our data for close (in feature space) instances and see what we can find:

In [10]:
prototypes = cf.get_counterfactuals(instance, n_counterfactuals=5, method="prototypes", desired_class=1)

In [11]:
instance

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
0,25,7,0.0,0.0,40.0,3,4,3,2,38,0.0,1.0,0


In [12]:
prototypes

,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
5953,20,8,0.0,0.0,35.0,3,4,3,2,38,0.0,1.0,1
24914,23,7,14344.0,0.0,40.0,3,4,3,1,39,0.0,1.0,1
7720,23,10,0.0,0.0,40.0,3,4,3,1,38,0.0,1.0,1
31169,36,7,13550.0,0.0,40.0,3,4,1,2,38,0.0,1.0,1
28800,33,10,8614.0,0.0,40.0,3,4,1,2,38,0.0,1.0,1


We see that there are some instances with different ages, capital gains or working hours per week. This would already provide some insights.
However, we see that some counterfactuals have a different race, native country and relationship status compared to our instance.

One may not want to change some features (relationship status) or may simply not be able to (race or native country).
For these cases, we can pass the parameter `fix_vars` that locks certain feature values: 

In [13]:
cf.get_counterfactuals(instance, n_counterfactuals=5, method="prototypes", desired_class=1, fix_vars=['relationship', 'race'])


/home/useradd/counterfactuals/src/cf_code/prototypes.py:145: UserWarning: only 4 counterfactuals found. k was set to 5, returning available counterfactuals:
  warnings.warn(


,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
5953,20,8,0.0,0.0,35.0,3,4,3,2,38,0.0,1.0,1
1900,30,9,99999.0,0.0,40.0,3,4,3,2,38,1.0,0.0,1
47392,22,10,99999.0,0.0,55.0,5,4,3,2,38,0.0,1.0,1
42771,39,10,0.0,0.0,50.0,3,2,3,2,38,1.0,0.0,1


We see that relationship status and race are now fixed. However, a different marital status or sex might be suggested. Now this may be exposing some bias in the model, data, or both, but we'll accept it for the moment and focus on te warning that was thrown: we see that5 counterfactuals were requested, but only 4 could be found in the data.

Now this would be a limit of this approach: it only returns solutions that are already known. But surely, there must be other combinations. There are. And don't call me Shirley.

We can use a genetic algorithm to try and generate data that may not be present already, but that is realistic enough and has the desired outcome. Links to the genetic algorithm used can be found in the README for this repository, but briefly, in order to ensure the generated data is realistic, we use 4 different fitness functions:

- *outcome fitness*: the predicted outcome of the generated data must be the one we want
- *sparsity fitness*: the least amount of features we change, the better
- *point likelihood fitness*: the generated point must be close to the data distribution of the actual data
- *distance fitness*: the generated instance must be close to the base instance in feature space

In [15]:
genetic = cf.get_counterfactuals(instance, 5, "genetic", desired_class=1, fix_vars=['relationship', 'race'])
genetic

Generation 7; counterfactuals found: 10/5


,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
0,32,13,12903.510184,0.000000,40.000000,1,4,3,2,38,0.0,1.000,0.998578
1,25,7,8618.769041,232.373006,29.805851,3,4,3,2,38,0.0,1.000,0.995547
2,25,7,5437.713931,127.590360,23.406455,3,2,3,2,38,0.0,0.875,0.716267
3,55,7,7574.310256,121.667161,24.214694,3,2,3,2,38,0.0,0.000,0.977984
4,25,7,7888.315837,127.147820,25.792370,3,4,3,2,38,0.0,1.000,0.662177


In this specific instance we see that we have `sex_Female` identically 0 and `sex_Male` being 0.875 or 0. These values are allowed because the one-hot encoded columns are float values (which is in itself an issue data-wise, though it is very modern of they/them), but also because we didn't explicitly say that these columns are one-hot encoded and should co-mutate.

The code was developed so that we only need to list the prefixes of one-hot encoded columns. In this case, `sex_`:

In [16]:
genetic = cf.get_counterfactuals(instance, 5, "genetic", desired_class=1, fix_vars=['relationship', 'race'], one_hot_encoded = ['sex_'])
genetic

Generation 9; counterfactuals found: 13/5


,age,education_num,capital_gain,capital_loss,hours_per_week,workclass,marital_status,relationship,race,native_country,sex_Female,sex_Male,outcome
0,19,7,12393.957351,0.000000,40.000000,3,2,3,2,38,0.0,1.0,0.986366
1,25,13,9339.921503,12.358000,35.352525,3,4,3,2,38,1.0,0.0,0.996720
2,23,7,4794.576589,130.799698,32.317489,3,4,3,2,38,0.0,1.0,0.731972
3,68,7,10472.339531,275.127051,13.282103,1,4,3,2,38,0.0,1.0,0.940040
4,25,7,8390.966104,260.412856,23.105424,3,2,3,2,38,1.0,0.0,0.884210


and we see the old-fashioned binary gender once again.

# Regression counterfactuals

Similarly, we can show an example for regression.

We use the `california-housing` dataset, trying to predict house prices (in units of $100k) in California based on some neighborhood data.

In this case, all features are numeric and no encoding was necessary.

In [2]:
X, y = load_dataset('california_housing', include_description=True)

{'AveBedrms': 'Average number of bedrooms per household',
 'AveOccup': 'Average house occupancy (household size)',
 'AveRooms': 'Average number of rooms per household',
 'HouseAge': 'Median house age in block group',
 'Latitude': 'Block group latitude',
 'Longitude': 'Block group longitude',
 'MedInc': 'Median income in block group',
 'Population': 'Block group population',
 'target': 'MedHouseVal, Median house value in 100k USD'}


In [3]:
X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


once again, no optimization happening when modelling. Just a plain XGBoostRegressor:

In [8]:
model = joblib.load(ARTIFACT_DIR / "xgboost_regressor.pkl")

model_metrics = Path(ARTIFACT_DIR / "regression_report.json")

with open(model_metrics) as f:
    model_metrics = json.load(f)
    
model_metrics

{'mse': 0.2167858596472864,
 'mae': 0.3030349616541927,
 'r2': 0.8374442572107926}

We see that the errors could be better, but, again, optimizing model performance is beyond the scope of this demo.

We go straight into picking a data instance:

In [9]:
instance = X.iloc[0:1].copy()
instance_outcome = y.iloc[0:1]
instance.loc[:, 'outcome'] = instance_outcome.values

instance

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,8.3252,41.0,6.984127,1.02381,322.0,2.555556,37.88,-122.23,4.526


so we see that a house with in this neighborhood would cost about 450k. Now, given the desire to buy, one might ask what type of data would give a price lower than 400k? Let's look at prototypes first:

In [10]:
cf = Counterfactuals(X=X, y=y, model=model)
prototypes = cf.get_counterfactuals(instance, n_counterfactuals=3, method='prototypes', upper_limit=4)
prototypes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
275,7.0875,41.0,6.593458,1.088785,626.0,2.925234,37.79,-122.18,2.407
417,7.8336,37.0,6.132597,0.925414,903.0,2.494475,37.90,-122.26,3.713
9939,7.5197,42.0,4.240000,0.720000,40.0,1.600000,38.22,-122.28,2.750


so, a neighborhood with less median income and smaller houses might be more appropriate.

Note how we only specified an upper bound to my desired outcome (`upper_limit`). Of course, we might not want to go too cheap either, concerned about the value of the property in 20 years.
This can be set as well:

In [11]:
prototypes = cf.get_counterfactuals(instance, n_counterfactuals=3, method='prototypes', upper_limit=4, lower_limit=3)
prototypes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
1635,7.7736,36.0,6.870968,1.032258,398.0,2.567742,37.88,-122.20,3.781
138,7.0175,37.0,6.982955,1.028409,420.0,2.386364,37.82,-122.20,3.667
421,7.5385,37.0,6.666667,1.015556,987.0,2.193333,37.89,-122.25,3.500


Or maybe we just want to sell and get a feeling for what would need to be different in order to get a higher value:

In [12]:
prototypes = cf.get_counterfactuals(instance, n_counterfactuals=10, method='prototypes', lower_limit=4.6)
prototypes

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
17159,8.2782,37.0,6.750000,1.010417,540.0,2.812500,37.40,-122.20,5.00001
18321,7.9029,37.0,6.247573,0.956311,510.0,2.475728,37.45,-122.13,5.00001
9401,8.9248,37.0,6.463087,0.973154,721.0,2.419463,37.92,-122.56,5.00001
17050,6.8879,44.0,8.717172,1.151515,281.0,2.838384,37.47,-122.28,5.00001
18285,8.3924,37.0,7.215517,1.017241,945.0,2.715517,37.37,-122.10,5.00001
9399,8.6340,48.0,5.615942,0.916667,641.0,2.322464,37.90,-122.56,4.63500
18358,8.5294,35.0,8.186508,1.055556,676.0,2.682540,37.36,-122.10,5.00001
18278,8.5888,35.0,8.056122,1.071429,570.0,2.908163,37.35,-122.07,5.00001
16941,6.9533,44.0,7.608025,1.012346,843.0,2.601852,37.55,-122.34,5.00001
18279,7.7068,35.0,7.126984,1.095238,548.0,2.899471,37.35,-122.08,5.00001


Of course, using genetic counterfactuals can also get more creative, though it might take a bit longer to get there:

In [16]:
prototypes = cf.get_counterfactuals(instance, n_counterfactuals=5, method='genetic', upper_limit=4)
prototypes

Generation 2; counterfactuals found: 16/5


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,outcome
0,8.325200,41.000000,6.984127,1.023810,322.000000,2.555556,69.515862,-122.230000,3.223096
1,1.542540,29.842764,4.967149,0.333575,2098.441639,2.555556,53.708968,-229.032668,0.603895
2,1.542540,29.842764,4.967149,0.997824,2098.441639,2.555556,53.708968,-175.627881,0.666982
3,1.589945,41.000000,6.984127,1.023810,322.000000,2.555556,37.880000,-122.230000,1.761659
4,1.326170,29.842764,2.583625,0.625938,2098.441639,2.555556,69.619570,-175.627881,0.721991


and everything else works exactly the same as in the regression case